In [1]:
import gc
import pickle
import numpy as np
import torch

def datato3d(arrays):#list of np arrays, NULL*3*100
    output=list()
    for i in arrays:
        i=np.transpose(i,(0,2,1))
        output.append(i)
    return output


def udflip(X_nparray, y_nparray, shuffle=True):

    if X_nparray.shape[2] == 4:
        if np.std(X_nparray[:, 0, :]) > np.std(X_nparray[:, -1, :]):
            print("Detected special info in first column, swapping...")
            X_nparray = np.concatenate((X_nparray[:, 1:, :], X_nparray[:, 0:1, :]), axis=1)
    
    X_flipped = np.flip(X_nparray, axis=2)  

    X_aug = np.vstack((X_nparray, X_flipped))
    y_aug = np.hstack((y_nparray, y_nparray))  

    if shuffle:
        shuffle_idx = np.random.permutation(X_aug.shape[0])
        return X_aug[shuffle_idx], y_aug[shuffle_idx]
    else:
        return X_aug, y_aug

with open('../data_61_26_ROIV2.pkl','rb') as f:
    data=pickle.load(f)


dataList=datato3d([data['X_train'],data['X_test']])



In [2]:
data['X_train'].shape, data['X_test'].shape,data['X_train'][4].shape

((3496641, 100, 4), (1497440, 100, 4), (100, 4))

In [3]:
data['X_train'][4]

array([[-50.86827  ,  -5.1718316, -11.912324 ,   0.       ],
       [-50.886658 ,  -5.4194384, -11.963684 ,   0.       ],
       [-50.911953 ,  -5.687157 , -12.018323 ,   0.       ],
       [-50.926834 ,  -5.954355 , -12.07456  ,   0.       ],
       [-50.913975 ,  -6.200402 , -12.130718 ,  20.       ],
       [-50.85936  ,  -6.430222 , -12.192032 ,  20.       ],
       [-50.769756 ,  -6.6747646, -12.26447  ,  20.       ],
       [-50.662407 ,  -6.9200387, -12.3379   ,  20.       ],
       [-50.554653 ,  -7.151108 , -12.401925 ,  20.       ],
       [-50.4517   ,  -7.367921 , -12.4543   ,  20.       ],
       [-50.331234 ,  -7.6009817, -12.509045 ,  20.       ],
       [-50.201836 ,  -7.8392177, -12.557598 ,  20.       ],
       [-50.074444 ,  -8.068485 , -12.589779 ,  20.       ],
       [-49.95623  ,  -8.280725 , -12.597824 ,  20.       ],
       [-49.827026 ,  -8.510864 , -12.589802 ,  20.       ],
       [-49.689793 ,  -8.750706 , -12.565458 ,  20.       ],
       [-49.55394  ,  -8

In [ ]:
X_train=dataList[0]
X_test=dataList[1]
# X_train, y_train = X_train[:, :, ::5], data['y_train'][:]
# X_test, y_test = X_test[:, :, ::5], data['y_test'][:]
X_train, y_train = X_train[:, :, ::1], data['y_train'][:]
X_test, y_test = X_test[:, :, ::1], data['y_test'][:]

y_test_list=data['y_test'].tolist()

NCLASS=max(y_test_list)+1

X_train,y_train=udflip(X_train,y_train,shuffle=True)
X_test,y_test=udflip(X_test,y_test,shuffle=False)

X_train=torch.from_numpy(X_train)#data['X_train'])
y_train=torch.from_numpy(y_train.astype(np.int32))#data['y_train'])

X_test=torch.from_numpy(X_test)
y_test=torch.from_numpy(y_test.astype(np.int32))  


In [ ]:

del data,dataList
gc.collect()
print('data loaded!')
print('X_train_shape',X_train.size())
print('X_test_shape',X_test.size())
print('classes: ', NCLASS)

In [ ]:
import torch.utils.data as utils
kwargs = {'num_workers': 1, 'pin_memory': True}
X_test = torch.from_numpy(X_test)
tst_set = utils.TensorDataset(X_test, y_test)
tst_loader = utils.DataLoader(tst_set, batch_size=1000, shuffle=False, **kwargs)

In [ ]:
import torch.utils.data as utils
X_test=tracks_data['tracks']['data']
X_test=rescale(X_test,int(tracks_data['tracks']['count']))
#X_test_ud=np.asarray(udflip(X_test,int(mat['tracks']['count']))).astype(np.float32)
X_test=np.asarray(X_test).astype(np.float32)
#X_test=np.vstack((X_test,X_test_ud))
#X_test=X_test.reshape((X_test.shape[0],1,X_test.shape[1],X_test.shape[2]))
#X_test=datato3d(X_test)[0]
X_test=np.transpose(X_test,(0,2,1))

In [ ]:
X_test[0].shape

In [ ]:
import h5py
import numpy as np

def loadmat(filename):
    '''
    读取 MATLAB v7.3 `.mat` 文件（Whole_tracks 作为 tracks）
    '''
    output = dict()
    
    # 打开 HDF5 MAT 文件
    with h5py.File(filename, 'r') as data:
        # 读取 Whole_tracks 变量
        if 'Whole_tracks' not in data:
            raise KeyError("❌ 错误: 'Whole_tracks' 变量不存在！")

        whole_tracks = data['Whole_tracks']  # 结构体 Whole_tracks

        # 确保它有 `count` 和 `data`
        if 'count' not in whole_tracks or 'data' not in whole_tracks:
            raise KeyError(f"❌ 错误: 'Whole_tracks' 结构不完整！包含: {list(whole_tracks.keys())}")

        # 读取 count（可能是字符编码格式，需要解析）
        count = whole_tracks['count'][()]  
        print("🔍 Whole_tracks['count'] 数据:", count)
        print("🔍 数据类型:", type(count))

        # 直接转换成整数
        total_count = int(count.item())
        # 读取 Whole_tracks['data']
        track = []
        for i in range(total_count):
            data_ref = whole_tracks['data'][i].item()
            track.append(np.transpose(data[data_ref][:]).astype(np.float32))

        # 组织输出
        output['tracks'] = {
            'count': total_count,
            'data': track
        }
    
    return output

# 运行测试
filename = "../Testing_Set/J0037_class_label.mat"
tracks_data = loadmat(filename)
print("✅ 成功读取 Whole_tracks 数据！")
print(f"轨迹数: {tracks_data['tracks']['count']}")


In [ ]:
import h5py

filename = "../Testing_Set/J0037_class_label.mat"

with h5py.File(filename, 'r') as f:
    print("MAT 文件变量:", list(f.keys()))  # 列出顶层变量
    
    for key in f.keys():
        print(f"\n🔍 变量 '{key}' 结构:")
        try:
            print("    子变量:", list(f[key].keys()))  # 如果是 group，打印内部结构
        except AttributeError:
            print("    不是结构体，可能是数组或标量")


In [ ]:
import h5py
import numpy as np

def load_label_mat(filename):
    """
    读取 MATLAB v7.3 (.mat) 文件中的 `class_label`
    """
    with h5py.File(filename, 'r') as data:
        if 'class_label' not in data:
            raise KeyError("❌ 错误: 'class_label' 变量不存在！")

        # 读取 class_label
        class_label = data['class_label'][()]

        # 解析数据
        if isinstance(class_label, np.ndarray):
            if class_label.size == 1:  # 只有一个值
                class_label = class_label.item()
            else:  # 多个值，转换为 NumPy 数组
                class_label = np.array(class_label)
        else:
            class_label = int(class_label)  # 可能是单个数值

        print(f"✅ 成功解析 class_label: {class_label}")
        return class_label

# 测试
filename = "../Testing_Set/J0037_class_label.mat"
labels = load_label_mat(filename)
print("📌 解析出的 labels:", labels)


In [55]:
import torch
import numpy as np
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support

def evaluate_model(model, test_loader, device='cuda'):
    """
    通用测试函数（无损失函数计算）
    - 适用于分类任务
    - 计算预测结果、混淆矩阵、Precision、Recall、F1

    参数：
    - model: 训练好的 PyTorch 模型
    - test_loader: PyTorch DataLoader (测试集)
    - device: 'cuda' or 'cpu'

    返回：
    - conf_matrix: 混淆矩阵
    - precision, recall, f1: 分类指标
    """
    model.eval()  # 设置模型为评估模式
    preds = []
    labels = []
    
    with torch.no_grad():  # 不计算梯度，加速推理
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)

            # 获取模型输出
            output = model(data)

            # 分类任务：获取预测类别
            if output.dim() > 1:  # 确保 output 是 logits 形式
                pred = output.argmax(dim=1)  # 取最大概率的类别
                preds.extend(pred.cpu().numpy())
                labels.extend(target.cpu().numpy())

    # 计算分类指标
    if preds:
        conf_matrix = confusion_matrix(labels, preds)
        precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    else:
        conf_matrix, precision, recall, f1 = None, None, None, None

    # 输出结果
    print(f'📊 Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}')
    print(f'🔢 混淆矩阵:\n{conf_matrix}')

    return conf_matrix, precision, recall, f1
